In [ ]:
## data processing ###
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torchvision.datasets as td
from torchvision import transforms
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import plot_confusion_matrix,f1_score
import torchvision
from torchvision import transforms
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as Data
import torchvision.datasets as td
from torchvision import transforms
import pathlib

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
drive = pathlib.Path('./drive/MyDrive') / 'ML_Project' / "Artwork-Detection-and-Style-Transfer-main"

In [ ]:
!ls

drive  sample_data


In [ ]:
### load information table##
data=pd.read_csv(drive/'dataset'/'label info.csv',sep='\t')
print(type(data))
del data['Unnamed: 0']
#data.head(50)
#data.rename(columns={"":"","Name":"Name","format":"materials", "creator":"creator","type":"type", "date":"date", "subject":"subject"})
all_creators = {}
for index, row in data.iterrows():
  m = row['format'][1:-2]
  tmlist = m.split(",")
  mlist = []
  for t in tmlist:
    if "materiaal" in t and t[13:-2] not in mlist:
      mlist.append(t[13:-2])
  data.at[index,'format'] = str(mlist)
  c = row['creator'][2:-2]
  t = c.split("', '")[0]
  p = t.find(":")
  t = t[p+2:]
  if t not in all_creators:
    all_creators[t] = [index]
  else:
    all_creators[t].append(index) 
  data.at[index,'creator'] = t
print(len(all_creators))
print(all_creators)
all_creators_filtered = {}
for key in all_creators:
  if len(all_creators[key])>=50 and key != 'anoniem':
    all_creators_filtered[key] = all_creators[key][:50]
print(all_creators_filtered)
data.head(60)

In [ ]:
len(all_creators_filtered)

367

In [ ]:
[['Lange', 'Johannes Philippus'],
 ['Bruyn', 'Nicolaes de'],
 ['anoniem'],
 ['anoniem'],
 ['Rembrandt Harmensz. van Rijn'],
 ['Debrie', 'Gabriel François Louis'],
 ['Houbraken', 'Jacob'],
 ['Goltzius', 'Hendrick'],
 ['Masanobu', 'Kitao'],

# Type

In [ ]:
### the whole size of database###
type_list=np.asarray(data.type)
#u_name=np.unique(type_l)
len(type_list)

NameError: ignored

In [ ]:
### choose 5 interesting label and test its size###
type1=data[data.type=='[\'bord (vaatwerk)\']'].shape[0] ## pottery##
type2=data[data.type=='[\'prent\']'].shape[0] ##print###
type3=data[data.type=='[\'tekening\']'].shape[0] ## painting##
type4=data[data.type=='[\'foto\']'].shape[0]  ## photo##
type5=data[data.type=='[\'beeldhouwwerk\']'].shape[0] ##sculpture##

print(f'{type1},{type2},{type3},{type4},{type5}')

In [ ]:
###create train dataset###
temp1=data[data.type=='[\'bord (vaatwerk)\']'].Name.values[:200] ##label 0
temp2=data[data.type=='[\'prent\']'].Name.values[:200]  ##label 1
temp3=data[data.type=='[\'tekening\']'].Name.values[:200] ##label 2
temp4=data[data.type=='[\'foto\']'].Name.values[:200] ## label 3
temp5=data[data.type=='[\'beeldhouwwerk\']'].Name.values[:200] ##label 4

sample_list=np.c_[list1,list2,list3,list4,list5]
sample_dataframe=pd.DataFrame(sample_list)
sample_dataframe.columns=[i for i in range(5)]
sample_dataframe.head(15)

In [ ]:
## use this set as evaluation set###
test1=data[data.type=='[\'bord (vaatwerk)\']'].Name.values[201:204] 
test2=data[data.type=='[\'prent\']'].Name.values[201:204] 
test3=data[data.type=='[\'tekening\']'].Name.values[201:204]
test4=data[data.type=='[\'foto\']'].Name.values[201:204]
test5=data[data.type=='[\'beeldhouwwerk\']'].Name.values[201:204]
test_list=np.c_[test1,test2,test3,test4,test5]
test_dataframe=pd.DataFrame(test_list)
test_dataframe.columns=[i for i in range(5)]

In [ ]:
### construct new test set###
###random choose 15 images per label###
np.random.seed(675)
##first label##
index_li1=np.where(data[data.type=='[\'bord (vaatwerk)\']'].Name.values)
label_f=data.Name.values[index_li1]
index_f=np.random.randint(0,519,15)
name_f=data[data.type=='[\'bord (vaatwerk)\']'].Name.values[index_f]
##second_label##
index_li2=np.where(data[data.type=='[\'prent\']'].Name.values)
label_s=data.Name.values[index_li2]
index_s=np.random.randint(0,52166,15)
name_s=data[data.type=='[\'prent\']'].Name.values[index_s]
### third_label##
index_li3=np.where(data[data.type=='[\'tekening\']'].Name.values)
label_t=data.Name.values[index_li3]
index_t=np.random.randint(0,10882,15)
name_t=data[data.type=='[\'tekening\']'].Name.values[index_t]
## fourth_label##
index_li4=np.where(data[data.type=='[\'foto\']'].Name.values)
label_fo=data.Name.values[index_li4]
index_fo=np.random.randint(0,1768,15)
name_fo=data[data.type=='[\'foto\']'].Name.values[index_fo]
###fif_label##
index_li5=np.where(data[data.type=='[\'beeldhouwwerk\']'].Name.values)
label_fif=data.Name.values[index_li5]
index_fif=np.random.randint(0,538,15)
name_fif=data[data.type=='[\'beeldhouwwerk\']'].Name.values[index_fif]

##construct combined name dataframe##
new_test=np.c_[name_f,name_s,name_t,name_fo,name_fif]
new_test_df=pd.DataFrame(new_test)
new_test_df.columns=[i for i in range(5)]

# Creator

In [ ]:
### the whole size of database###
creator_list=np.asarray(data.creator)
#u_name=np.unique(type_l)
len(creator_list)

NameError: ignored

In [ ]:
### choose 5 interesting label and test its size###
type1=data[data.type=='[\'bord (vaatwerk)\']'].shape[0] ## pottery##
type2=data[data.type=='[\'prent\']'].shape[0] ##print###
type3=data[data.type=='[\'tekening\']'].shape[0] ## painting##
type4=data[data.type=='[\'foto\']'].shape[0]  ## photo##
type5=data[data.type=='[\'beeldhouwwerk\']'].shape[0] ##sculpture##

print(f'{type1},{type2},{type3},{type4},{type5}')

519,52166,10882,1768,538


In [ ]:
###create train dataset###
temp1=data[data.type=='[\'bord (vaatwerk)\']'].Name.values[:200] ##label 0
temp2=data[data.type=='[\'prent\']'].Name.values[:200]  ##label 1
temp3=data[data.type=='[\'tekening\']'].Name.values[:200] ##label 2
temp4=data[data.type=='[\'foto\']'].Name.values[:200] ## label 3
temp5=data[data.type=='[\'beeldhouwwerk\']'].Name.values[:200] ##label 4

sample_list=np.c_[list1,list2,list3,list4,list5]
sample_dataframe=pd.DataFrame(sample_list)
sample_dataframe.columns=[i for i in range(5)]
sample_dataframe.head(15)

In [ ]:
## use this set as evaluation set###
test1=data[data.type=='[\'bord (vaatwerk)\']'].Name.values[201:204] 
test2=data[data.type=='[\'prent\']'].Name.values[201:204] 
test3=data[data.type=='[\'tekening\']'].Name.values[201:204]
test4=data[data.type=='[\'foto\']'].Name.values[201:204]
test5=data[data.type=='[\'beeldhouwwerk\']'].Name.values[201:204]
test_list=np.c_[test1,test2,test3,test4,test5]
test_dataframe=pd.DataFrame(test_list)
test_dataframe.columns=[i for i in range(5)]

In [ ]:
### construct new test set###
###random choose 15 images per label###
np.random.seed(675)
##first label##
index_li1=np.where(data[data.type=='[\'bord (vaatwerk)\']'].Name.values)
label_f=data.Name.values[index_li1]
index_f=np.random.randint(0,519,15)
name_f=data[data.type=='[\'bord (vaatwerk)\']'].Name.values[index_f]
##second_label##
index_li2=np.where(data[data.type=='[\'prent\']'].Name.values)
label_s=data.Name.values[index_li2]
index_s=np.random.randint(0,52166,15)
name_s=data[data.type=='[\'prent\']'].Name.values[index_s]
### third_label##
index_li3=np.where(data[data.type=='[\'tekening\']'].Name.values)
label_t=data.Name.values[index_li3]
index_t=np.random.randint(0,10882,15)
name_t=data[data.type=='[\'tekening\']'].Name.values[index_t]
## fourth_label##
index_li4=np.where(data[data.type=='[\'foto\']'].Name.values)
label_fo=data.Name.values[index_li4]
index_fo=np.random.randint(0,1768,15)
name_fo=data[data.type=='[\'foto\']'].Name.values[index_fo]
###fif_label##
index_li5=np.where(data[data.type=='[\'beeldhouwwerk\']'].Name.values)
label_fif=data.Name.values[index_li5]
index_fif=np.random.randint(0,538,15)
name_fif=data[data.type=='[\'beeldhouwwerk\']'].Name.values[index_fif]

##construct combined name dataframe##
new_test=np.c_[name_f,name_s,name_t,name_fo,name_fif]
new_test_df=pd.DataFrame(new_test)
new_test_df.columns=[i for i in range(5)]

# Creator and Material

In [ ]:
import pickle 
with open( drive / "dataset" / "label.pickle", 'rb') as pickle_file:
    data_ = pickle.load(pickle_file)

## creator

In [ ]:
#extract sample index
names = data_['identifier']
sample_index = []
evaluation_index = []
new_test_index = []

for id in sample_dataframe.values.ravel():
  num = id[8:-4]
  sample_index.append(names.index(num))

for id in test_dataframe.values.ravel():
  num = id[8:-4]
  evaluation_index.append(names.index(num))

for id in new_test_df.values.ravel():
  num = id[8:-4]
  new_test_index.append(names.index(num))

NameError: ignored

In [ ]:
#creator
creators_ = data_["creator"]
sample_creators = []
evaluation_creators = []
new_test_creators = [] 

for id in sample_index:
  sample_creators.append(creators_[id])

for id in evaluation_index:
  evaluation_creators.append(creators_[id])

for id in new_test_index:
  new_test_creators.append(creators_[id])

# Multi-task

In [ ]:
import pickle 

with open( drive / "dataset" / "label.pickle", 'rb') as pickle_file:
    data = pickle.load(pickle_file)
pickle_file.close()

In [ ]:
data['material']

In [ ]:
data['creator']

In [ ]:
data['type']

In [ ]:
data['identifier']

In [ ]:
import operator

In [ ]:
creator = {}
creator_freq = {}
for i,c in enumerate(data['creator']):
  if c not in creator_freq.keys():
    creator_freq[c]= 0
    creator[c]=[]
  creator_freq[c] += 1
  creator[c].append(data['identifier'][i])
creator_keys = dict(sorted(creator_freq.items(), key=operator.itemgetter(1), reverse=True)[:17])
print(creator_keys)
creator_keys = creator_keys.keys()
creator_filtered = {}
C_images = []
for c in creator_keys:
  if c != '' and c != 'anoniem':
    creator_filtered[c] = creator[c]
    C_images = C_images + list(set(creator[c]) - set(C_images)) 

{'anoniem': 16518, 'Luyken, Jan': 1928, 'Picart, Bernard': 1504, 'Rembrandt Harmensz. van Rijn': 1384, 'Fokke, Simon': 1267, 'Houbraken, Jacob': 1148, 'Hooghe, Romeyn de': 1119, 'Callot, Jacques': 1013, 'Meissener Porzellan Manufaktur': 1003, 'Gheyn, Jacob de (II)': 893, 'Passe, Crispijn van de (I)': 856, 'Galle, Philips': 838, 'Goltzius, Hendrick': 810, '': 685, 'Collaert, Adriaen': 649, 'Hogenberg, Frans': 640, 'Vinkeles, Reinier': 623}


In [ ]:
creator_freq

In [ ]:
creator_filtered

In [ ]:
material = {}
material_freq = {}
for i,r in enumerate(data['material']):
  for m in r:
    if m not in material_freq.keys():
      material_freq[m] = 0
      material[m] = []
    material_freq[m] += 1
    material[m].append(data['identifier'][i])
material_keys = dict(sorted(material_freq.items(), key=operator.itemgetter(1), reverse=True)[:15])
print(material_keys)
material_keys = material_keys.keys()
material_filtered = {}
M_images = []
for c in material_keys:
  material_filtered[c] = material[c]
  M_images = M_images + list(set(material[c]) - set(M_images)) 

{'papier': 92785, 'olieverf': 3667, 'porselein': 2811, 'hout': 2164, 'zilver': 1788, 'doek': 1753, 'paneel': 1588, 'glazuur': 1423, 'faience': 1307, 'inkt': 1038, 'ijzer': 1032, 'karton': 1027, 'waterverf': 950, 'fotopapier': 933, 'messing': 921}


In [ ]:
material_freq

In [ ]:
material_filtered

In [ ]:
Type = {}
Type_freq = {}
for i,r in enumerate(data['type']):
  for m in r:
    if m not in Type_freq.keys():
      Type_freq[m] = 0
      Type[m] = []
    Type_freq[m] += 1
    Type[m].append(data['identifier'][i])
Type_keys = dict(sorted(Type_freq.items(), key=operator.itemgetter(1), reverse=True)[:15])
print(Type_keys)
Type_keys = Type_keys.keys()
Type_filtered = {}
T_images = []
for c in Type_keys:
    Type_filtered[c] = Type[c]
    T_images = T_images + list(set(Type[c]) - set(T_images))  

{'prent': 77052, 'tekening': 14223, 'boekillustratie': 7007, 'ornamentprent': 5221, 'schilderij': 3593, 'historieprent': 3503, 'nieuwsprent': 2598, 'foto': 2319, 'ontwerp': 1966, 'kaart': 1923, 'titelpagina': 1570, 'spotprent': 1029, 'tekstblad': 920, 'bord (vaatwerk)': 898, 'eet- en drinkgerei': 839}


In [ ]:
Type_keys

dict_keys(['prent', 'tekening', 'boekillustratie', 'ornamentprent', 'schilderij', 'historieprent', 'nieuwsprent', 'foto', 'ontwerp', 'kaart', 'titelpagina', 'spotprent', 'tekstblad', 'bord (vaatwerk)', 'eet- en drinkgerei'])

In [ ]:
Type_filtered

In [ ]:
images = list(set(C_images) - (set(C_images) - set(T_images)))
images = list(set(images) - (set(images) - set(M_images)))

In [ ]:
len(images)

15261

In [ ]:
import random

random.shuffle(images)
images_train = images[:int(len(images)*0.7)]
images_eval = images[int(len(images)*0.7):int(len(images)*0.8)]
images_test = images[int(len(images)*0.8):]

In [ ]:
len(images_train)

10682

In [ ]:
with open( drive / "dataset" / "images.pickle", 'wb') as pickle_file:
    pickle.dump(images, pickle_file)
pickle_file.close()

In [ ]:
image_data_train = {}
image_data_train['identifier'] = []
image_data_train['material'] = []
image_data_train['creator'] = []
image_data_train['type'] = []
for image in images_train:
  image_data_train['identifier'].append(image)
  i = data['identifier'].index(image)
  image_data_train['material'].append(data['material'][i])
  image_data_train['creator'].append(data['creator'][i])
  image_data_train['type'].append(data['type'][i])

In [ ]:
image_data_eval = {}
image_data_eval['identifier'] = []
image_data_eval['material'] = []
image_data_eval['creator'] = []
image_data_eval['type'] = []
for image in images_eval:
  image_data_eval['identifier'].append(image)
  i = data['identifier'].index(image)
  image_data_eval['material'].append(data['material'][i])
  image_data_eval['creator'].append(data['creator'][i])
  image_data_eval['type'].append(data['type'][i])

In [ ]:
image_data_test = {}
image_data_test['identifier'] = []
image_data_test['material'] = []
image_data_test['creator'] = []
image_data_test['type'] = []
for image in images_test:
  image_data_test['identifier'].append(image)
  i = data['identifier'].index(image)
  image_data_test['material'].append(data['material'][i])
  image_data_test['creator'].append(data['creator'][i])
  image_data_test['type'].append(data['type'][i])

In [ ]:
image_data_train

In [ ]:
with open( drive / "dataset" / "train_label.pickle", 'wb') as pickle_file:
    pickle.dump(image_data_train, pickle_file)
pickle_file.close()

In [ ]:
with open( drive / "dataset" / "eval_label.pickle", 'wb') as pickle_file:
    pickle.dump(image_data_eval, pickle_file)
pickle_file.close()

In [ ]:
with open( drive / "dataset" / "test_label.pickle", 'wb') as pickle_file:
    pickle.dump(image_data_test, pickle_file)
pickle_file.close()

# Pls don't run the code below for now, they may cause unwanted change to  'label.pickle'.

In [ ]:
creator = ''
for i in range(len(data['creator'])):
  if len(data['creator'][i]) > 1:
    for j in data['creator'][i][:-1]:
      creator = j+', '
    creator += data['creator'][i][-1]
    data['creator'][i] = creator
  else:
    data['creator'][i] = data['creator'][i][0]

In [ ]:
data['creator']

In [ ]:
for i in range(len(data['identifier'])):
  data['identifier'][i] = data['identifier'][i]

In [ ]:
data['identifier']

In [ ]:
with open( drive / "dataset" / "label.pickle", 'wb') as pickle_file:
  pickle.dump(data, pickle_file)
pickle_file.close()


# Store

In [ ]:
#type
sample_dataframe.to_csv(drive / "dataset" / "label" / "type" / "sample_type_label.csv")
test_dataframe.to_csv(drive / "dataset" / "label" / "type" / "val_type_label.csv")
new_test_df.to_csv(drive / "dataset" / "label" / "type" / "test_type_label.csv")

In [ ]:
result=pd.read_csv(drive / "dataset" / "label" / "type" / 'sample_type_label.csv',sep='\,')
type(result.iloc[0,0])

numpy.int64